In [136]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn.preprocessing
from sklearn.metrics import r2_score

from keras.layers import Dense,Dropout,SimpleRNN,LSTM
from keras.models import Sequential


In [137]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [138]:
train.head()

,userID_hash,month,x0,x1,x2,x3,x4,x5,x6,x7,...,previous_x46,previous_x47,previous_x48,previous_x49,previous_x50,previous_x51,previous_x52,previous_x53,previous_x54,y
0,2d90a44b2d37900619c2398ee3b07c24ae45c9436fb44b...,2022-09,4.000000,4.0,0.0,2.0,2.0,0.0,138000.0,0.0,...,0.0,2.0,2.0,0.0,61000.0,0.0,30500.0,0.0,1.0,177000.0
1,097a3239adb67f3d00fe75338a7e1082363746384dff00...,2022-07,2.090909,11.0,1.0,8.0,8.0,1.0,7087542.0,750000.0,...,0.0,1.0,1.0,0.0,180300.0,0.0,90150.0,0.0,1.0,14893793.0
2,9f9f56fdc788f9a817ec9928309921539a47f6656fe7d6...,2022-07,9.333333,4.0,0.0,3.0,3.0,0.0,267050.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232200.0
3,b5ccb95b3a44bc48b5d8bd819cf48689f756c5c2669619...,2022-10,0.500000,19.0,4.0,7.0,7.0,1.0,1756880.0,453000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410200.0
4,afd48246a21834776f945fc30e10ea7bba1d051b01798d...,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105000.0


In [139]:
dt = train[['y', 'previous_x50']]

In [140]:
dt = train.drop(['userID_hash'], axis = 1)

In [141]:
dt[['year', 'month']] = dt['month'].str.split('-', expand=True)

In [142]:
dt.head()

,month,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,previous_x47,previous_x48,previous_x49,previous_x50,previous_x51,previous_x52,previous_x53,previous_x54,y,year
0,09,4.000000,4.0,0.0,2.0,2.0,0.0,138000.0,0.0,34500.000,...,2.0,2.0,0.0,61000.0,0.0,30500.0,0.0,1.0,177000.0,2022
1,07,2.090909,11.0,1.0,8.0,8.0,1.0,7087542.0,750000.0,590628.500,...,1.0,1.0,0.0,180300.0,0.0,90150.0,0.0,1.0,14893793.0,2022
2,07,9.333333,4.0,0.0,3.0,3.0,0.0,267050.0,0.0,66762.500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232200.0,2022
3,10,0.500000,19.0,4.0,7.0,7.0,1.0,1756880.0,453000.0,76386.086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410200.0,2022
4,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105000.0,2022


In [143]:
dt.drop(['year'], axis =  1, inplace=True)

In [144]:
dt_cr = dt.corr()

In [145]:
# import seaborn as sns
# sns.heatmap(dt)

In [146]:
dt_cr.head()

,month,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,previous_x46,previous_x47,previous_x48,previous_x49,previous_x50,previous_x51,previous_x52,previous_x53,previous_x54,y
month,1.000000,-0.037993,0.006870,0.002134,-0.011399,-0.011656,0.006694,0.001326,0.001184,-0.000038,...,-0.041991,-0.010853,-0.007868,-0.058440,0.020340,-0.019364,0.038010,-0.017446,0.042982,0.002304
x0,-0.037993,1.000000,-0.154549,-0.086340,-0.120357,-0.119229,-0.108257,-0.001757,-0.006251,0.015679,...,-0.006919,-0.128300,-0.128384,-0.007913,-0.073419,-0.004446,-0.047516,-0.000096,-0.003491,-0.042220
x1,0.006870,-0.154549,1.000000,0.215506,0.687310,0.695710,0.272375,0.134086,0.138636,0.028534,...,0.031259,0.441043,0.441118,0.038327,0.257551,0.018030,-0.020195,-0.001795,0.005937,0.240748
x2,0.002134,-0.086340,0.215506,1.000000,0.242690,0.125212,0.766013,0.029867,0.094979,-0.018904,...,0.042763,0.057216,0.056016,0.036431,0.035997,0.017845,-0.009936,0.012926,-0.030996,0.025979
x3,-0.011399,-0.120357,0.687310,0.242690,1.000000,0.980493,0.345495,0.009260,0.021295,-0.028199,...,0.022758,0.593517,0.593798,0.033583,0.187741,0.009512,-0.056344,-0.003733,0.010104,0.193882


In [147]:
result = dt_cr[dt_cr['y'] > 0.3]

In [148]:
result.reset_index()

,index,month,x0,x1,x2,x3,x4,x5,x6,x7,...,previous_x46,previous_x47,previous_x48,previous_x49,previous_x50,previous_x51,previous_x52,previous_x53,previous_x54,y
0,x28,-0.001176,-0.054442,0.320027,0.056667,0.249176,0.253779,0.089149,0.724060,0.171205,...,0.011406,0.173307,0.173354,0.017882,0.569440,0.086665,0.277768,0.030460,0.005909,0.747832
1,x30,0.004221,-0.008458,-0.045140,-0.029180,-0.084444,-0.079010,-0.042933,0.339813,0.081826,...,-0.008315,-0.062241,-0.062041,-0.008645,0.312792,0.036853,0.543534,0.031818,0.005228,0.370403
2,x39,0.004194,-0.050751,0.164536,0.022668,0.161011,0.169166,0.040080,0.598492,0.165322,...,0.004136,0.037206,0.037018,0.008586,0.148058,0.011414,0.103679,0.008616,-0.000419,0.736163
3,x41,0.016177,0.014457,-0.031540,-0.069580,-0.068779,-0.053151,-0.095030,0.303890,0.066004,...,-0.000971,0.012832,0.012713,0.001092,0.143100,0.019270,0.158223,0.014877,-0.001118,0.379226
4,x50,0.016079,-0.078990,0.356884,0.053705,0.242004,0.245501,0.084921,0.710198,0.141619,...,0.012268,0.217916,0.218165,0.018166,0.707062,0.114779,0.346861,0.040951,0.010924,0.578179
5,previous_x28,-0.001583,-0.054319,0.236069,0.038726,0.196466,0.199149,0.068364,0.543213,0.125028,...,0.021276,0.241919,0.241875,0.028294,0.794169,0.093356,0.382262,0.028293,0.010543,0.683253
6,previous_x30,0.017526,-0.008503,-0.047116,-0.020088,-0.083373,-0.080641,-0.026980,0.245770,0.086189,...,-0.024154,-0.058435,-0.057298,-0.025989,0.456274,0.017305,0.901068,0.011138,0.032283,0.353437
7,previous_x39,0.005615,-0.044259,0.136426,0.020307,0.139420,0.143594,0.046089,0.490769,0.128072,...,0.007540,0.038266,0.037899,0.010006,0.158227,0.013753,0.088881,0.006350,0.003503,0.676977
8,previous_x41,0.033846,-0.003162,-0.034852,-0.032556,-0.065052,-0.058249,-0.042589,0.239740,0.101120,...,-0.010625,0.009566,0.009785,-0.008601,0.110062,0.008888,0.115024,0.007046,0.013000,0.363969
9,previous_x50,0.020340,-0.073419,0.257551,0.035997,0.187741,0.189996,0.061058,0.494756,0.098227,...,0.022670,0.311187,0.311448,0.029717,1.000000,0.117191,0.484693,0.032254,0.015534,0.507635


In [149]:
index = []
for i in result.index:
    index.append(i)
print(index)

['x28', 'x30', 'x39', 'x41', 'x50', 'previous_x28', 'previous_x30', 'previous_x39', 'previous_x41', 'previous_x50', 'y']


In [150]:
df = dt[index]

In [151]:
df.shape

(1777360, 11)

In [152]:
df.head()

,x28,x30,x39,x41,x50,previous_x28,previous_x30,previous_x39,previous_x41,previous_x50,y
0,69000.0,34500.00,NaN,NaN,69000.0,61000.0,30500.0,NaN,NaN,61000.0,177000.0
1,5487542.0,609726.90,3720542.0,744108.4,1767000.0,9353167.0,935316.7,9172867.0,1146608.4,180300.0,14893793.0
2,267050.0,66762.50,267050.0,66762.5,NaN,NaN,NaN,NaN,NaN,NaN,232200.0
3,980880.0,70062.86,NaN,NaN,980880.0,NaN,NaN,NaN,NaN,NaN,410200.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105000.0


In [153]:
df['month'] = dt['month']

C:\Users\ductr\AppData\Local\Temp\ipykernel_29936\222916567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = dt['month']


In [154]:
df.isna().sum()

x28              662434
x30              662434
x39             1307384
x41             1307384
x50              926935
previous_x28     763634
previous_x30     763634
previous_x39    1308166
previous_x41    1308166
previous_x50    1027696
y                     0
month                 0
dtype: int64

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1777360 entries, 0 to 1777359
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   x28           float64
 1   x30           float64
 2   x39           float64
 3   x41           float64
 4   x50           float64
 5   previous_x28  float64
 6   previous_x30  float64
 7   previous_x39  float64
 8   previous_x41  float64
 9   previous_x50  float64
 10  y             float64
 11  month         object 
dtypes: float64(11), object(1)
memory usage: 162.7+ MB


In [156]:
df.isna().sum()

x28              662434
x30              662434
x39             1307384
x41             1307384
x50              926935
previous_x28     763634
previous_x30     763634
previous_x39    1308166
previous_x41    1308166
previous_x50    1027696
y                     0
month                 0
dtype: int64

In [157]:
df.head()

,x28,x30,x39,x41,x50,previous_x28,previous_x30,previous_x39,previous_x41,previous_x50,y,month
0,69000.0,34500.00,NaN,NaN,69000.0,61000.0,30500.0,NaN,NaN,61000.0,177000.0,09
1,5487542.0,609726.90,3720542.0,744108.4,1767000.0,9353167.0,935316.7,9172867.0,1146608.4,180300.0,14893793.0,07
2,267050.0,66762.50,267050.0,66762.5,NaN,NaN,NaN,NaN,NaN,NaN,232200.0,07
3,980880.0,70062.86,NaN,NaN,980880.0,NaN,NaN,NaN,NaN,NaN,410200.0,10
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105000.0,10


In [158]:
df = df.fillna(0)

In [159]:
df.isna().sum()

x28             0
x30             0
x39             0
x41             0
x50             0
previous_x28    0
previous_x30    0
previous_x39    0
previous_x41    0
previous_x50    0
y               0
month           0
dtype: int64

In [160]:
df

,x28,x30,x39,x41,x50,previous_x28,previous_x30,previous_x39,previous_x41,previous_x50,y,month
0,69000.0,34500.000,0.0,0.0,69000.0,61000.0,30500.0,0.0,0.0,61000.0,177000.0,09
1,5487542.0,609726.900,3720542.0,744108.4,1767000.0,9353167.0,935316.7,9172867.0,1146608.4,180300.0,14893793.0,07
2,267050.0,66762.500,267050.0,66762.5,0.0,0.0,0.0,0.0,0.0,0.0,232200.0,07
3,980880.0,70062.860,0.0,0.0,980880.0,0.0,0.0,0.0,0.0,0.0,410200.0,10
4,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,105000.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...
1777355,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20000.0,09
1777356,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000.0,09
1777357,65000.0,21666.666,0.0,0.0,65000.0,0.0,0.0,0.0,0.0,0.0,65000.0,10
1777358,980.0,980.000,0.0,0.0,980.0,0.0,0.0,0.0,0.0,0.0,115296.0,10


In [161]:
# from sklearn.preprocessing import MinMaxScaler
# min = MinMaxScaler()
# data_scale = min.fit_transform(df)

In [162]:
df['y'].value_counts().sort_values()

y
222969.0          1
2867068.0         1
7767983.0         1
1353866.0         1
737235.0          1
              ...  
50000.0       27544
5000.0        31029
10000.0       67228
20000.0       70447
0.0          209701
Name: count, Length: 444208, dtype: int64

In [163]:
x= df.drop(['y'],axis=1,inplace=False).values
y= df['y'].values

In [164]:
y

array([  177000., 14893793.,   232200., ...,    65000.,   115296.,
         108000.])

In [165]:
import sklearn

In [166]:
def normalize_data(df):
    scaler = sklearn.preprocessing.MinMaxScaler()
    df['y']=scaler.fit_transform(df['y'].values.reshape(-1,1))
    return df

df_norm = normalize_data(df)
df_norm.shape

(1777360, 12)

In [170]:
df_norm.head()
df_norm.drop(['month'],axis=1,inplace=True)

In [172]:
df_norm.head()

,x28,x30,x39,x41,x50,previous_x28,previous_x30,previous_x39,previous_x41,previous_x50,y
0,69000.0,34500.00,0.0,0.0,69000.0,61000.0,30500.0,0.0,0.0,61000.0,0.000347
1,5487542.0,609726.90,3720542.0,744108.4,1767000.0,9353167.0,935316.7,9172867.0,1146608.4,180300.0,0.029202
2,267050.0,66762.50,267050.0,66762.5,0.0,0.0,0.0,0.0,0.0,0.0,0.000455
3,980880.0,70062.86,0.0,0.0,980880.0,0.0,0.0,0.0,0.0,0.0,0.000804
4,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000206


In [176]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Scaling the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_norm)

# Converting to 3D shape for LSTM [samples, timesteps, features]
X = scaled_data[:, :-1]  # All columns except 'y'
y = scaled_data[:, -1]   # Only 'y' column

X = X.reshape((X.shape[0], 1, X.shape[1]))  # Assuming one timestep

print(X.shape, y.shape)

(1777360, 1, 10) (1777360,)


In [177]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Step 4 and 5: Build, Train and Evaluate the LSTM model
    # ...


In [178]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

input_shape = (X_train.shape[1], X_train.shape[2])

model = build_lstm_model(input_shape)
history = model.fit(X_train, y_train, epochs=50, batch_size=1, validation_data=(X_test, y_test), verbose=2)


Epoch 1/50


c:\Users\ductr\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, Callback, ModelCheckpoint

# Merge inputs and targets
inputs = np.concatenate((TRAIN_IMAGES, TRAIN_VAL), axis=0)
targets = np.concatenate((TEST_IMAGES, TEST_VAL), axis=0)
acc_per_fold = []
loss_per_fold = []
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', 
                  data_format='channels_last', input_shape=(28,28,1)))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', 
                  data_format='channels_last'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', 
                  data_format='channels_last'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))      
  model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', strides=1, padding='same', 
                  data_format='channels_last'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(128, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  early_stopping_callback = EarlyStopping(monitor='val_loss', 
  #                                         min_delta=1e-2,
                                          patience=4,
                                          verbose = 1,
                                          restore_best_weights=True)

  ckpt = ModelCheckpoint(
          f'modelcheck{fold_no}.weights.h5',
          verbose=1,
          save_best_only=True,
          save_weights_only=True
      )
  optimizer = Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999 )

  model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  history = model.fit(TRAIN_IMAGES,TEST_IMAGES,
                  batch_size=128,
                  epochs=10,
                  validation_data=(TRAIN_VAL,TEST_VAL),
  #                  validation_steps = TRAIN_VAL.shape[0] // 64,
                  verbose=1,
                  callbacks = [early_stopping_callback,ckpt])  
  scores = model.evaluate(TRAIN_VAL, TEST_VAL)
#   model.save('model.h5')
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
 
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1  